In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import scipy.optimize as sco
import json

In [2]:
all_positions = pd.DataFrame()
tickers = ['AMZN', 'BA', 'CAT', 'GOOGL', 'GS', 'NKE', 'NVDA', 'SOFI', 'TSLA', 'UNH']
for stock in tickers:
    all_positions[stock] = pd.read_csv('Predictions of ' + stock + '.csv', parse_dates=True, index_col = 0)['pos']
    
all_positions.fillna(0, inplace=True)
print(all_positions)


            AMZN   BA  CAT  GOOGL   GS  NKE  NVDA  SOFI  TSLA  UNH
Date                                                              
2024-03-01  -0.5 -0.5  1.0    1.0 -0.5 -0.5  -0.5  -0.5   0.0  1.0
2024-03-04  -0.5 -0.5  1.0    1.0  1.0 -0.5  -0.5  -0.5   0.0  1.0
2024-03-05  -0.5 -0.5  1.0    1.0  1.0 -0.5  -0.5  -0.5   0.0 -0.5
2024-03-06  -0.5  1.0  1.0    1.0  1.0 -0.5  -0.5  -0.5   0.0 -0.5
2024-03-07  -0.5  1.0  1.0    1.0  1.0 -0.5  -0.5  -0.5   0.0 -0.5
...          ...  ...  ...    ...  ...  ...   ...   ...   ...  ...
2025-01-10  -0.5  1.0  1.0    1.0  1.0  1.0  -0.5   1.0  -0.5  1.0
2025-01-13  -0.5 -0.5  1.0    1.0  1.0  1.0  -0.5   1.0  -0.5 -0.5
2025-01-14  -0.5 -0.5  1.0    1.0  1.0 -0.5  -0.5   1.0  -0.5  1.0
2025-01-15  -0.5  1.0  1.0    1.0 -0.5  1.0  -0.5   1.0  -0.5 -0.5
2025-01-16  -0.5 -0.5  1.0    1.0 -0.5 -0.5  -0.5   1.0  -0.5 -0.5

[221 rows x 10 columns]


In [3]:
# Parameters
train_start = "2021-01-01"
test_start = "2024-03-01"
test_end = "2025-01-16"
initial_capital = 50000
lookback_days = 60

# Download both Close and Open prices
data = yf.download(tickers, start=train_start, end=test_end, progress=False)
close = data['Close'].dropna()
openp = data['Open'].dropna()

# Log returns (from close prices)
log_returns = np.log(close / close.shift(1)).dropna()

# Generate monthly rebalance dates starting from test_start
rebalance_dates = pd.date_range(start=test_start, end=test_end, freq='MS')  # Month Start
# Map to actual trading days (forward-fill if it's not a trading day)
rebalance_dates = [close.index[close.index.get_indexer([d], method='bfill')[0]] for d in rebalance_dates]


# Init
capital = initial_capital
portfolio_history = []
weights_per_month = {}
shares_held = pd.Series(0, index=tickers)

for date in rebalance_dates:
    # Ensure date exists in index
    if date not in close.index:
        date = close.index[close.index.get_indexer([date], method='bfill')[0]]
    
    end_idx = close.index.get_loc(date)
    start_idx = end_idx - lookback_days
    if start_idx < 0:
        continue

    # Get past data window (only up to yesterday)
    window_returns = log_returns.iloc[start_idx:end_idx]
    mean_returns = window_returns.mean() * 252
    cov_matrix = window_returns.cov() * 252

    # Define Sharpe Ratio optimizer
    def neg_sharpe(weights):
        port_return = np.dot(weights, mean_returns.values)
        port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix.values, weights)))
        return -port_return / port_vol

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = [(0.01, 0.3)] * len(tickers)
    init_guess = np.array([1 / len(tickers)] * len(tickers))

    result = sco.minimize(neg_sharpe, init_guess, method='SLSQP',
                          bounds=bounds, constraints=constraints)
    if not result.success:
        continue

    weights = pd.Series(result.x, index=tickers)

    # Log weights
    weights_per_month[pd.to_datetime(date).strftime("%Y-%m-%d")] = weights.round(4).to_dict()

    # ---- 🔒 Avoid Lookahead Bias Here ----
    # Execute trades using NEXT DAY's OPEN prices
    next_day_idx = close.index.get_loc(date) + 1
    if next_day_idx >= len(close):
        break  # no next day, skip

    next_day = close.index[next_day_idx]
    prices = openp.loc[next_day]  # Use Open price of next day

    # Update portfolio
    target_value = weights * capital
    new_shares = (target_value / prices).fillna(0).astype(int)
    cost = (new_shares * prices).sum()
    leftover_cash = capital - cost

    shares_held = new_shares
    capital = (shares_held * prices).sum() + leftover_cash
    portfolio_history.append({'Date': next_day, 'Portfolio Value': capital})

# Results
results = pd.DataFrame(portfolio_history).set_index('Date')
print(results)
print(f"\n📈 Final Portfolio Value on {test_end}: ${capital:,.2f}")

print("\n📊 Diversified Weights per Month:")
print(json.dumps(weights_per_month, indent=2))


YF.download() has changed argument auto_adjust default to True
            Portfolio Value
Date                       
2024-03-04          50000.0
2024-04-02          50000.0
2024-05-02          50000.0
2024-06-04          50000.0
2024-07-02          50000.0
2024-08-02          50000.0
2024-09-04          50000.0
2024-10-02          50000.0
2024-11-04          50000.0
2024-12-03          50000.0
2025-01-03          50000.0

📈 Final Portfolio Value on 2025-01-16: $50,000.00

📊 Diversified Weights per Month:
{
  "2024-03-01": {
    "AMZN": 0.1136,
    "BA": 0.01,
    "CAT": 0.3,
    "GOOGL": 0.01,
    "GS": 0.2264,
    "NKE": 0.01,
    "NVDA": 0.3,
    "SOFI": 0.01,
    "TSLA": 0.01,
    "UNH": 0.01
  },
  "2024-04-01": {
    "AMZN": 0.01,
    "BA": 0.01,
    "CAT": 0.3,
    "GOOGL": 0.1032,
    "GS": 0.2368,
    "NKE": 0.01,
    "NVDA": 0.3,
    "SOFI": 0.01,
    "TSLA": 0.01,
    "UNH": 0.01
  },
  "2024-05-01": {
    "AMZN": 0.01,
    "BA": 0.01,
    "CAT": 0.1152,
    "GOOGL": 0.3,
 

/Users/vanyabendatu/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/vanyabendatu/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/vanyabendatu/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/vanyabendatu/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/vanyabend

In [6]:
predictions = {}
for stock in tickers:
    predictions[stock] = pd.read_csv('Predictions of ' + stock + '.csv', parse_dates=True, index_col=0)
#combined_predictions = pd.concat(predictions.values(), axis=1, keys=predictions.keys())
# print(predictions)
# Print the combined DataFrame
#print(combined_predictions)
    
def calculate_portolio_value():
    portfolio_val = 50000

    # Sorted list of rebalance dates
    rebalance_dates = list(weights_per_month.keys())

    for i in range(len(rebalance_dates)):
        date = rebalance_dates[i]
        curr_portfolio_value = portfolio_val
        for stock in tickers:
            
            next_date = None
            if i == len(rebalance_dates) - 1:
                next_date = '2025-01-16'
            else:
                next_date = rebalance_dates[i+1]
                
            w = weights_per_month[date][stock]
            portfolio_val += w * curr_portfolio_value * (predictions[stock]['Cumulative_Return_Strategy'].loc[next_date]-predictions[stock]['Cumulative_Return_Strategy'].loc[date])
            
    return portfolio_val

In [7]:
calculate_portolio_value()

100565.55157363591